In [3]:
%load_ext autoreload
%autoreload 2

In [12]:
%reload_ext autoreload

In [2]:
import warnings
warnings.filterwarnings('ignore')
from optimizers_niapy import *
from other_optimizers import *

Nodes: 21557, Edges: 342353


# GCN results

In [13]:
model = GCNLinkPredictor(in_channels=5, hidden_channels=256)
optimizer = optim.Adam(model.parameters(), lr=0.01)

print("\nNo optimizing")
epoch, loss, auc_gcn, f1_gcn, ndcg = 0,0,0,0,0
for _ in range(1, 10):
  loss = GCNtrain(model, optimizer, train_data)
  test_probs = GCNtest(model, test_data)
  auc_gcn, f1_gcn, ndcg = evaluate_model(test_probs, test_data.edge_label)
  epoch=_
print(f"Epoch {epoch+1}, Loss: {loss:.4f}, AUC score: {auc_gcn:.4f}, F1 score: {f1_gcn:.4f}, NDCG: {ndcg:.4f}")


No optimizing
Epoch 10, Loss: 1.2861, AUC score: 0.8775, F1 score: 0.8071, NDCG: 0.9875


# GAN results

In [14]:
generator = Generator(in_channels=5, hidden_channels=256).to(device)
discriminator = Discriminator(hidden_channels=256).to(device)

generator.apply(init_weights)
discriminator.apply(init_weights)

optimizer_G = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.5, 0.999))

print("\nNo optimizing")
epoch, d_loss, g_loss, auc_gan, f1_gan, ndcg = 0,0,0,0,0,0
for _ in range(1, 10):
    d_loss, g_loss = GANtrain(generator, discriminator, optimizer_G, optimizer_D, train_data)
    test_probs = GANtest(generator, discriminator, test_data)
    auc_gan, f1_gan, ndcg = evaluate_model(test_probs, test_data.edge_label)
    epoch=_
print(f"Epoch {epoch+1}, Average Loss: {(d_loss-g_loss):.4f}, AUC score: {auc_gan:.4f}, F1 score: {f1_gan:.4f}, NDCG: {ndcg:.4f}")



No optimizing
Epoch 10, Average Loss: -0.3044, AUC score: 0.7528, F1 score: 0.7337, NDCG: 0.9699


# Results after optimization with Genetic Algorithm

## GCN

In [10]:
result = run_gcn_ga()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 73
  Learning Rate   : 0.01220
  Num Layers      : 3
  Dropout         : 0.40

Performance Metrics:
  F1 Score        : 0.8506
  AUC             : 0.9125
  Average Loss    : 1.2876
  NDCG            : 0.9913


## GAN

In [6]:
result = run_gan_ga()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 459
  Learning Rate   : 0.00190
  Dropout         : 0.33

Performance Metrics:
  F1 Score        : 0.7538
  AUC             : 0.7772
  Average Loss    : 0.0088
  NDCG            : 0.9723


# Results after optimization with Particle Swarm Optimization

## GCN

In [12]:
result = run_gcn_pso()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 107
  Learning Rate   : 0.01339
  Num Layers      : 3
  Dropout         : 0.55

Performance Metrics:
  F1 Score        : 0.8506
  AUC             : 0.7790
  Average Loss    : 1.3920
  NDCG            : 0.9680


## GAN

In [11]:
result = run_gan_pso()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 512
  Learning Rate   : 0.00200
  Dropout         : 0.31

Performance Metrics:
  F1 Score        : 0.7571
  AUC             : 0.7781
  Average Loss    : 0.0147
  NDCG            : 0.9706


# Results after optimization with Simulated Annealing

## GCN

In [6]:
result = run_gcn_sa()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 160
  Learning Rate   : 0.00974
  Num Layers      : 3
  Dropout         : 0.23

Performance Metrics:
  F1 Score        : 0.8435
  AUC             : 0.7841
  Average Loss    : 1.3751
  NDCG            : 0.9685


## GAN

In [12]:
result = run_gan_sa()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 412
  Learning Rate   : 0.00200
  Dropout         : 0.10

Performance Metrics:
  F1 Score        : 0.7584
  AUC             : 0.7583
  Average Loss    : -0.0260
  NDCG            : 0.9699


# Results after optimization with Artificial Bee Colony

## GCN

In [7]:
result = run_gcn_abc()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 106
  Learning Rate   : 0.00691
  Num Layers      : 3
  Dropout         : 0.15

Performance Metrics:
  F1 Score        : 0.8496
  AUC             : 0.8831
  Average Loss    : 1.4161
  NDCG            : 0.9885


## GAN

In [6]:
result = run_gan_abc()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 480
  Learning Rate   : 0.00136
  Dropout         : 0.22

Performance Metrics:
  F1 Score        : 0.7545
  AUC             : 0.7773
  Average Loss    : 0.0421
  NDCG            : 0.9724


# Results after optimization with Ant Colony Optimization

## GCN

In [7]:
result = run_gcn_aco(train_data, test_data)

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 224
  Learning Rate   : 0.00215
  Num Layers      : 3
  Dropout         : 0.70

Performance Metrics:
  F1 Score        : 0.8419
  AUC             : 0.9145
  Average Loss    : 1.2698
  NDCG            : 0.9918


## GAN

In [8]:
result=run_gan_aco(train_data, test_data)

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 384
  Learning Rate   : 0.00003
  Dropout         : 0.30

Performance Metrics:
  F1 Score        : 0.7167
  AUC             : 0.7353
  Average Loss    : 0.0170
  NDCG            : 0.9649


# Results after optimization with Hill Climbing

## GCN

In [10]:
result = run_gcn_hc()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 244
  Learning Rate   : 0.01102
  Num Layers      : 3
  Dropout         : 0.66

Performance Metrics:
  F1 Score        : 0.8430
  AUC             : 0.9109
  Average Loss    : 1.3705
  NDCG            : 0.9914


## GAN

In [11]:
result = run_gan_hc()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 393
  Learning Rate   : 0.00200
  Dropout         : 0.40

Performance Metrics:
  F1 Score        : 0.7559
  AUC             : 0.7773
  Average Loss    : 0.0015
  NDCG            : 0.9726


# Results after optimization with Random Search

## GCN

In [8]:
result = run_gcn_ra()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 38
  Learning Rate   : 0.01378
  Num Layers      : 3
  Dropout         : 0.18

Performance Metrics:
  F1 Score        : 0.8493
  AUC             : 0.9147
  Average Loss    : 1.2535
  NDCG            : 0.9920


## GAN

In [13]:
result = run_gan_ra()

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 367
  Learning Rate   : 0.00166
  Dropout         : 0.20

Performance Metrics:
  F1 Score        : 0.7541
  AUC             : 0.7752
  Average Loss    : 0.0432
  NDCG            : 0.9725


# Results after optimization with Bayesian Optimization

## GCN

In [9]:
result = run_gcn_bo(train_data, test_data)

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 80
  Learning Rate   : 0.00785
  Num Layers      : 4
  Dropout         : 0.10

Performance Metrics:
  F1 Score        : 0.8504
  AUC             : 0.9154
  Average Loss    : 1.2674
  NDCG            : 0.9911


## GAN

In [13]:
result = run_gan_bo(train_data, test_data)

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 144
  Learning Rate   : 0.00062
  Dropout         : 0.35

Performance Metrics:
  F1 Score        : 0.7505
  AUC             : 0.7743
  Average Loss    : 0.2257
  NDCG            : 0.9725


# Results after optimizing with Optuna

## GCN

In [14]:
result = run_gcn_optuna(train_data, test_data)

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")

[I 2025-06-29 13:17:23,751] A new study created in memory with name: no-name-44c9f718-c8ae-4c7a-a5d0-7c2f787c941a
[I 2025-06-29 13:17:52,791] Trial 0 finished with value: -0.8489819563958007 and parameters: {'hidden_channels': 168, 'lr': 0.0018471485405569912, 'num_layers': 4, 'dropout': 0.06068083910325634}. Best is trial 0 with value: -0.8489819563958007.
[I 2025-06-29 13:18:20,060] Trial 1 finished with value: -0.6666666666666666 and parameters: {'hidden_channels': 183, 'lr': 0.05753480511986514, 'num_layers': 3, 'dropout': 0.40349376435931106}. Best is trial 0 with value: -0.8489819563958007.
[I 2025-06-29 13:18:31,361] Trial 2 finished with value: -0.7115093693753382 and parameters: {'hidden_channels': 63, 'lr': 0.00018248535688806364, 'num_layers': 2, 'dropout': 0.49167641182959493}. Best is trial 0 with value: -0.8489819563958007.
[I 2025-06-29 13:19:05,810] Trial 3 finished with value: -0.8033068729768572 and parameters: {'hidden_channels': 183, 'lr': 0.0006040468838495976, 'nu


Best Hyperparameters Found:
  Hidden Channels : 227
  Learning Rate   : 0.00153
  Num Layers      : 4
  Dropout         : 0.00

Performance Metrics:
  F1 Score        : 0.8512
  AUC             : 0.9166
  Average Loss    : 1.2522
  NDCG            : 0.9920


## GAN

In [15]:
result = run_gan_optuna(train_data, test_data)

print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")

[I 2025-06-29 13:39:46,542] A new study created in memory with name: no-name-95a22fe9-a275-406e-9f6a-fd53292054c7
[I 2025-06-29 13:41:23,959] Trial 0 finished with value: -0.6978073349426611 and parameters: {'hidden_channels': 512, 'lr': 1.745321089149644e-05, 'dropout': 0.419628882862882}. Best is trial 0 with value: -0.6978073349426611.
[I 2025-06-29 13:42:45,528] Trial 1 finished with value: -0.7431369056028896 and parameters: {'hidden_channels': 417, 'lr': 6.497490893937845e-05, 'dropout': 0.3845932918109063}. Best is trial 1 with value: -0.7431369056028896.
[I 2025-06-29 13:43:29,346] Trial 2 finished with value: -0.7120844168403859 and parameters: {'hidden_channels': 227, 'lr': 5.622059544939263e-05, 'dropout': 0.4718102875685303}. Best is trial 1 with value: -0.7431369056028896.
[I 2025-06-29 13:43:45,272] Trial 3 finished with value: -0.7214562961427271 and parameters: {'hidden_channels': 163, 'lr': 5.9870510592092906e-05, 'dropout': 0.08994247619545441}. Best is trial 1 with v


Best Hyperparameters Found:
  Hidden Channels : 373
  Learning Rate   : 0.00068
  Dropout         : 0.51

Performance Metrics:
  F1 Score        : 0.7560
  AUC             : 0.7817
  Average Loss    : 0.2407
  NDCG            : 0.9738


# Results after optimizing with GridSearchCV

## GCN

In [7]:
result = run_gcn_gs(train_data, test_data)
print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Num Layers      : {result['best_params']['num_layers']}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 64
  Learning Rate   : 0.01000
  Num Layers      : 3
  Dropout         : 0.30

Performance Metrics:
  F1 Score        : 0.8497
  AUC             : 0.9151
  Average Loss    : 1.2031
  NDCG            : 0.9920


## GAN

In [6]:
result = run_gan_gs(train_data, test_data)
print("\nBest Hyperparameters Found:")
print(f"  Hidden Channels : {result['best_params']['hidden_channels']}")
print(f"  Learning Rate   : {result['best_params']['lr']:.5f}")
print(f"  Dropout         : {result['best_params']['dropout']:.2f}")

print("\nPerformance Metrics:")
print(f"  F1 Score        : {result['f1']:.4f}")
print(f"  AUC             : {result['auc']:.4f}")
print(f"  Average Loss    : {result['loss']:.4f}")
print(f"  NDCG            : {result['ndcg']:.4f}")


Best Hyperparameters Found:
  Hidden Channels : 512
  Learning Rate   : 0.00010
  Dropout         : 0.00

Performance Metrics:
  F1 Score        : 0.7428
  AUC             : 0.7630
  Average Loss    : 0.0554
  NDCG            : 0.9712
